# Домашнее задание 2
Барабанщиков Лев Романович

В этом задании продолжаем работу с датасетом lenta-ru-news для задачи классификации текстов по топикам, но используем word2vec эмбеддинги.


## 1. Установка random seed и импорт библиотек


In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import random
import gc
import warnings

# NLP библиотеки
import nltk
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm.notebook import tqdm

# Эмбеддинги
from gensim.models import Word2Vec
import navec

# ML библиотеки
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

# Обработка данных
from corus import load_lenta

warnings.filterwarnings('ignore')

# Установка random seed для воспроизводимости результатов
RANDOM_SEED = 777
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

print("Random seed установлен на значение:", RANDOM_SEED)
print("Все библиотеки импортированы успешно!")


Random seed установлен на значение: 777
Все библиотеки импортированы успешно!


## 2. Загрузка и предобработка данных

Переиспользуем данные из ДЗ1 и подготовим их для работы с эмбеддингами


In [2]:
# Загрузка данных из ДЗ1 (переиспользуем подготовленные данные)
path = '../hw1/lenta-ru-news.csv.gz'
records = load_lenta(path)

print("Данные загружены!")

# Преобразуем в список
data = []
for record in tqdm(records, desc="Обработка записей"):
    if record.topic is None:
        continue
    data.append({
        'title': record.title,
        'text': record.text,
        'topic': record.topic
    })

print(f"Загружено {len(data)} записей")


Данные загружены!


Обработка записей: 0it [00:00, ?it/s]

Загружено 739351 записей


In [3]:
# Создаем DataFrame
df = pd.DataFrame(data)

# Вывод информации о датасете
print("Размер датасета:", len(df))
print("Количество уникальных топиков:", df['topic'].nunique())
print("\nТоп-10 самых популярных топиков:")
print(df['topic'].value_counts().head(10))

# Объединяем title и text
df['content'] = df['title'] + ' ' + df['text']

print("\nПример записи:")
print("Топик:", df.iloc[0]['topic'])
print("Содержание:", df.iloc[0]['content'][:200] + "...")


Размер датасета: 739351
Количество уникальных топиков: 24

Топ-10 самых популярных топиков:
topic
Россия             160519
Мир                136680
Экономика           79538
Спорт               64421
Культура            53803
Бывший СССР         53402
Наука и техника     53136
Интернет и СМИ      44675
Из жизни            27611
Дом                 21734
Name: count, dtype: int64

Пример записи:
Топик: Россия
Содержание: Названы регионы России с самой высокой смертностью от рака Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, с...


## 3. Предобработка текстов

Подготовка текстов для обучения word2vec эмбеддингов и классификации


In [4]:
# Загрузка NLTK данных
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Инициализация морфологического анализатора и стоп-слов
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

print("Инструменты предобработки готовы!")


Инструменты предобработки готовы!


[nltk_data] Downloading package punkt to /Users/bitcoin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bitcoin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/bitcoin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
def preprocess_text(text):
    """
    Предобработка текста: очистка, токенизация, лемматизация
    Возвращает список токенов для обучения эмбеддингов
    """
    # Очистка текста
    text = re.sub(r'[^а-яёa-z\s]|\d+', '', text.lower())
    
    # Токенизация
    tokens = word_tokenize(text, language="russian")
    
    # Лемматизация и фильтрация
    tokens = [morph.parse(token)[0].normal_form 
              for token in tokens 
              if token not in stop_words and len(token) > 2]
    
    return tokens

def preprocess_text_string(text):
    """
    Предобработка текста с возвращением строки (для совместимости с ДЗ1)
    """
    tokens = preprocess_text(text)
    return ' '.join(tokens)

print("Функции предобработки определены!")

# Тест функции
test_text = "Это тестовый текст, который мы будем обрабатывать для проверки!"
print("\nТест предобработки:")
print(f"Исходный текст: {test_text}")
print(f"Токены: {preprocess_text(test_text)}")
print(f"Строка: {preprocess_text_string(test_text)}")


Функции предобработки определены!

Тест предобработки:
Исходный текст: Это тестовый текст, который мы будем обрабатывать для проверки!
Токены: ['это', 'тестовый', 'текст', 'который', 'быть', 'обрабатывать', 'проверка']
Строка: это тестовый текст который быть обрабатывать проверка


## 4. Разделение датасета на обучающую, валидационную и тестовую выборки (60/20/20)

**Обоснование:** Используем стратификацию для сохранения пропорций классов в каждой выборке. Пропорция 60/20/20 обеспечивает достаточно данных для обучения эмбеддингов и корректную валидацию моделей.


In [6]:
# Берем подвыборку для ускорения работы (можно увеличить для полного датасета)
sample_size = 100000  # Уменьшаем для демонстрации
df_sample = df.sample(n=min(sample_size, len(df)), random_state=RANDOM_SEED)

print(f"Работаем с выборкой размером: {len(df_sample)} записей")

# Инициализация pandarallel для параллельной обработки
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=8, verbose=1)

# Предобработка текстов с использованием параллелизма
print("Предобрабатываем тексты с использованием параллелизма...")

# Параллельная обработка токенов
df_sample['processed_tokens'] = df_sample['content'].parallel_apply(preprocess_text)

# Параллельная обработка строк
df_sample['processed_content'] = df_sample['content'].parallel_apply(preprocess_text_string)

# Фильтруем пустые тексты
df_sample = df_sample[df_sample['processed_tokens'].apply(len) > 0]
print(f"После фильтрации пустых текстов: {len(df_sample)} записей")


Работаем с выборкой размером: 100000 записей
Предобрабатываем тексты с использованием параллелизма...


После фильтрации пустых текстов: 100000 записей


In [7]:
# Подготовка данных для разделения
print("Анализ распределения классов...")
class_counts = df_sample['topic'].value_counts()
print(f"Всего классов: {len(class_counts)}")
print(f"Классы с минимальным количеством образцов:")
print(class_counts.tail(10))

# Фильтрация классов с малым количеством образцов (менее 6 для корректной стратификации)
min_samples_per_class = 6
valid_classes = class_counts[class_counts >= min_samples_per_class].index
print(f"\nКлассы с достаточным количеством образцов (>={min_samples_per_class}): {len(valid_classes)}")

# Фильтруем датасет
df_filtered = df_sample[df_sample['topic'].isin(valid_classes)].copy()
print(f"Размер отфильтрованного датасета: {len(df_filtered)} (было {len(df_sample)})")

# Разделение на train/valid/test (60/20/20)
X = df_filtered['processed_content']
y = df_filtered['topic']
X_tokens = df_filtered['processed_tokens']

print(f"\nПроводим стратифицированное разделение...")

# Первое разделение: 60% train, 40% остальное
X_train, X_temp, y_train, y_temp, X_tokens_train, X_tokens_temp = train_test_split(
    X, y, X_tokens, test_size=0.4, random_state=RANDOM_SEED, stratify=y
)

# Второе разделение: 20% valid, 20% test
X_valid, X_test, y_valid, y_test, X_tokens_valid, X_tokens_test = train_test_split(
    X_temp, y_temp, X_tokens_temp, test_size=0.5, random_state=RANDOM_SEED, stratify=y_temp
)

print(f"\nРазмеры выборок:")
print(f"Train: {len(X_train)} ({len(X_train)/len(df_filtered)*100:.1f}%)")
print(f"Valid: {len(X_valid)} ({len(X_valid)/len(df_filtered)*100:.1f}%)")
print(f"Test: {len(X_test)} ({len(X_test)/len(df_filtered)*100:.1f}%)")

print(f"\nТоп-5 классов в train выборке:")
print(y_train.value_counts().head())

print(f"\nПроверка стратификации - доли классов в выборках:")
train_proportions = y_train.value_counts(normalize=True).sort_index()
valid_proportions = y_valid.value_counts(normalize=True).sort_index()
test_proportions = y_test.value_counts(normalize=True).sort_index()

for class_name in train_proportions.index[:5]:  # Показываем первые 5 классов
    print(f"{class_name}: train={train_proportions[class_name]:.3f}, "
          f"valid={valid_proportions[class_name]:.3f}, "
          f"test={test_proportions[class_name]:.3f}")

# Очистка памяти
del df_sample, df_filtered, data, records, df
gc.collect()
print("\nПамять очищена!")


Анализ распределения классов...
Всего классов: 21
Классы с минимальным количеством образцов:
topic
Ценности          1031
Бизнес            1015
Путешествия        904
69-я параллель     167
Крым                94
Культпросвет        50
                    26
Легпром             15
Библиотека           7
МедНовости           1
Name: count, dtype: int64

Классы с достаточным количеством образцов (>=6): 20
Размер отфильтрованного датасета: 99999 (было 100000)

Проводим стратифицированное разделение...

Размеры выборок:
Train: 59999 (60.0%)
Valid: 20000 (20.0%)
Test: 20000 (20.0%)

Топ-5 классов в train выборке:
topic
Россия         13146
Мир            11068
Экономика       6440
Спорт           5193
Бывший СССР     4409
Name: count, dtype: int64

Проверка стратификации - доли классов в выборках:
: train=0.000, valid=0.000, test=0.000
69-я параллель: train=0.002, valid=0.002, test=0.002
Библиотека: train=0.000, valid=0.000, test=0.000
Бизнес: train=0.010, valid=0.010, test=0.010
Бывший СС

## 5. Обучение Word2Vec эмбеддингов - 2 балла

**Обоснование гиперпараметров:**
- `vector_size=100` - размер векторов эмбеддингов. 100 измерений достаточно для русского языка
- `window=5` - размер контекстного окна. 5 слов вокруг целевого слова обеспечивает хороший баланс между локальным и глобальным контекстом  
- `min_count=5` - минимальная частота слова. Фильтрует редкие слова, которые могут внести шум
- `workers=4` - количество потоков для ускорения обучения
- `sg=0` - используем CBOW (Continuous Bag of Words), который лучше работает на больших датасетах
- `epochs=10` - количество эпох обучения для получения качественных эмбеддингов


In [8]:
# Подготовка корпуса для обучения Word2Vec
# Объединяем все токены из обучающей выборки
train_corpus = X_tokens_train.tolist()

print(f"Размер корпуса для обучения: {len(train_corpus)} текстов")
print(f"Пример токенов: {train_corpus[0][:10]}")

# Создание и обучение Word2Vec модели
w2v_model = Word2Vec(
    sentences=train_corpus,
    vector_size=100,     # размер векторов
    window=5,            # размер контекстного окна
    min_count=5,         # минимальная частота слова
    workers=4,           # количество потоков
    sg=0,                # CBOW алгоритм
    epochs=10,           # количество эпох
    seed=RANDOM_SEED     # параметр для воспроизводимости (не random_state!)
)

print("Word2Vec модель обучена!")
print(f"Размер словаря: {len(w2v_model.wv.key_to_index)}")
print(f"Размер векторов: {w2v_model.vector_size}")


Размер корпуса для обучения: 59999 текстов
Пример токенов: ['награждение', 'премия', 'область', 'качество', 'сервис', 'товар', 'состояться', 'июнь', 'церемония', 'награждение']
Word2Vec модель обучена!
Размер словаря: 50533
Размер векторов: 100


In [9]:
# Визуальная оценка качества эмбеддингов

print("=== Оценка качества Word2Vec эмбеддингов ===")

# 1. Поиск похожих слов
print("\n1. Наиболее похожие слова:")
test_words = ['россия', 'президент', 'экономика', 'спорт', 'культура']
for word in test_words:
    if word in w2v_model.wv:
        similar = w2v_model.wv.most_similar(word, topn=5)
        print(f"  {word}: {[w[0] for w in similar]}")
    else:
        print(f"  {word}: слово не найдено в словаре")

# 2. Проверка семантических отношений (аналогии)
print("\n2. Семантические аналогии:")
try:
    # Пример: мужчина относится к королю, как женщина к ?
    result = w2v_model.wv.most_similar(positive=['женщина', 'король'], negative=['мужчина'], topn=3)
    print(f"  мужчина -> король, женщина -> {[w[0] for w in result]}")
except:
    print("  Не удалось найти аналогию для данного примера")

try:
    # Москва относится к России, как Париж к ?
    result = w2v_model.wv.most_similar(positive=['париж', 'россия'], negative=['москва'], topn=3)
    print(f"  москва -> россия, париж -> {[w[0] for w in result]}")
except:
    print("  Не удалось найти аналогию для данного примера")

# 3. Поиск лишнего слова
print("\n3. Поиск лишнего слова:")
test_groups = [
    ['россия', 'украина', 'беларусь', 'яблоко'],
    ['футбол', 'баскетбол', 'хоккей', 'политика'],
    ['экономика', 'финанс', 'деньги', 'спорт']
]

for group in test_groups:
    try:
        # Проверяем, есть ли все слова в словаре
        if all(word in w2v_model.wv for word in group):
            odd_one = w2v_model.wv.doesnt_match(group)
            print(f"  {group} -> лишнее: {odd_one}")
        else:
            missing = [w for w in group if w not in w2v_model.wv]
            print(f"  {group} -> слова {missing} не найдены в словаре")
    except:
        print(f"  {group} -> не удалось определить лишнее слово")


=== Оценка качества Word2Vec эмбеддингов ===

1. Наиболее похожие слова:
  россия: ['российский', 'украина', 'белоруссия', 'молдавия', 'страна']
  президент: ['президентский', 'премьерминистр', 'посол', 'экспрезидент', 'госсекретарь']
  экономика: ['экономический', 'ввп', 'конкурентоспособность', 'рецессия', 'авторынок']
  спорт: ['мутко', 'спортивный', 'футбол', 'рспорт', 'культура']
  культура: ['образование', 'туризм', 'просвещение', 'здравоохранение', 'природопользование']

2. Семантические аналогии:
  мужчина -> король, женщина -> ['монарх', 'правитель', 'георг']
  москва -> россия, париж -> ['франция', 'германия', 'италия']

3. Поиск лишнего слова:
  ['россия', 'украина', 'беларусь', 'яблоко'] -> лишнее: яблоко
  ['футбол', 'баскетбол', 'хоккей', 'политика'] -> лишнее: политика
  ['экономика', 'финанс', 'деньги', 'спорт'] -> слова ['деньги'] не найдены в словаре


## 6. Загрузка предобученных эмбеддингов - 1 балл

Загружаем предобученные русские эмбеддинги navec для сравнения с нашими word2vec эмбеддингами.


In [10]:
# Загрузка предобученных эмбеддингов
print("Загрузка предобученных эмбеддингов...")

navec_model = None

# Попробуем несколько способов загрузки navec
try:
    # Способ 1: автоматическая загрузка через wget
    print("Пытаемся скачать navec автоматически...")
    import subprocess
    import os
    
    navec_url = 'https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar'
    navec_file = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
    
    if not os.path.exists(navec_file):
        subprocess.run(['wget', '-O', navec_file, navec_url], check=True)
        print(f"Navec скачан в {navec_file}")
    
    navec_model = navec.Navec.load(navec_file)
    print(f"Navec загружен успешно!")
    print(f"Размер словаря: {len(navec_model.vocab.words)}")
    print(f"Размер векторов: {navec_model.pq.dim}")

except Exception as e1:
    print(f"Способ 1 не работает: {e1}")
    
    # Способ 2: использование requests для загрузки
    try:
        print("Пытаемся загрузить через requests...")
        import requests
        
        if not os.path.exists(navec_file):
            response = requests.get(navec_url, stream=True)
            response.raise_for_status()
            
            with open(navec_file, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Navec скачан через requests")
        
        navec_model = navec.Navec.load(navec_file)
        print(f"Navec загружен успешно!")
        print(f"Размер словаря: {len(navec_model.vocab.words)}")
        print(f"Размер векторов: {navec_model.pq.dim}")
        
    except Exception as e2:
        print(f"Способ 2 не работает: {e2}")
        
        # Способ 3: использование альтернативных предобученных эмбеддингов
        try:
            print("Пытаемся загрузить более простую версию или создать заглушку...")
            
            # Можно попробовать другую версию navec или создать заглушку
            print("Navec недоступен, будем работать только с Word2Vec эмбеддингами")
            navec_model = None
            
        except Exception as e3:
            print(f"Все способы не работают: {e3}")
            navec_model = None

if navec_model is None:
    print("\n⚠️ Navec эмбеддинги недоступны.")
    print("Будем продолжать работу только с нашими Word2Vec эмбеддингами.")
    print("Для полноценного сравнения рекомендуется установить navec эмбеддинги вручную:")


Загрузка предобученных эмбеддингов...
Пытаемся скачать navec автоматически...
Navec загружен успешно!
Размер словаря: 500002
Размер векторов: 300


In [11]:
print(f"\nСтатус navec: {'Загружен' if navec_model is not None else 'Недоступен'}")



Статус navec: Загружен


In [12]:
# Тестирование navec эмбеддингов
if navec_model is not None:
    print("\n=== Тестирование navec эмбеддингов ===")
    
    # Проверяем наличие тестовых слов
    test_words = ['россия', 'президент', 'экономика', 'спорт', 'культура']
    for word in test_words:
        if word in navec_model:
            print(f"Слово '{word}' найдено в navec")
        else:
            print(f"Слово '{word}' НЕ найдено в navec")
    
    # Тестовый вектор
    if 'россия' in navec_model:
        test_vector = navec_model['россия']
        print(f"\nВектор для слова 'россия': размер {test_vector.shape}")
        print(f"Первые 5 значений: {test_vector[:5]}")
else:
    print("Navec модель недоступна")



=== Тестирование navec эмбеддингов ===
Слово 'россия' найдено в navec
Слово 'президент' найдено в navec
Слово 'экономика' найдено в navec
Слово 'спорт' найдено в navec
Слово 'культура' найдено в navec

Вектор для слова 'россия': размер (300,)
Первые 5 значений: [ 0.22543699 -0.39721358  0.6805563   0.21706595 -0.19716908]


## 7. Сравнение эмбеддингов с LogisticRegression - 2 балла

Сравним качество классификации с разными эмбеддингами


In [13]:
# Функции для векторизации текстов с помощью эмбеддингов

def vectorize_text_w2v(tokens, model):
    """Векторизация текста с помощью Word2Vec (усредняем векторы слов)"""
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # Если ни одно слово не найдено, возвращаем нулевой вектор
        return np.zeros(model.vector_size)

def vectorize_text_navec(tokens, model):
    """Векторизация текста с помощью navec (усредняем векторы слов)"""
    if model is None:
        return np.zeros(300)  # Стандартный размер navec
    
    vectors = []
    for token in tokens:
        if token in model:
            vectors.append(model[token])
    
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.pq.dim)

def vectorize_corpus(tokens_list, model, method='w2v'):
    """Векторизация корпуса текстов"""
    vectors = []
    
    if method == 'w2v':
        for tokens in tqdm(tokens_list, desc=f"Векторизация {method}"):
            vectors.append(vectorize_text_w2v(tokens, model))
    elif method == 'navec':
        for tokens in tqdm(tokens_list, desc=f"Векторизация {method}"):
            vectors.append(vectorize_text_navec(tokens, model))
    
    return np.array(vectors)

print("Функции векторизации определены!")


Функции векторизации определены!


In [14]:
# Векторизация данных с помощью разных эмбеддингов

print("=== Векторизация данных ===")

# 1. Векторизация с помощью Word2Vec
print("\n1. Word2Vec векторизация...")
X_train_w2v = vectorize_corpus(X_tokens_train, w2v_model, 'w2v')
X_valid_w2v = vectorize_corpus(X_tokens_valid, w2v_model, 'w2v')

print(f"Размер train набора (w2v): {X_train_w2v.shape}")
print(f"Размер valid набора (w2v): {X_valid_w2v.shape}")

# 2. Векторизация с помощью navec (если доступен)
if navec_model is not None:
    print("\n2. Navec векторизация...")
    X_train_navec = vectorize_corpus(X_tokens_train, navec_model, 'navec')
    X_valid_navec = vectorize_corpus(X_tokens_valid, navec_model, 'navec')
    
    print(f"Размер train набора (navec): {X_train_navec.shape}")
    print(f"Размер valid набора (navec): {X_valid_navec.shape}")
else:
    print("\n2. Navec недоступен, пропускаем...")
    X_train_navec = None
    X_valid_navec = None

print("\nВекторизация завершена!")


=== Векторизация данных ===

1. Word2Vec векторизация...


Векторизация w2v:   0%|          | 0/59999 [00:00<?, ?it/s]

Векторизация w2v:   0%|          | 0/20000 [00:00<?, ?it/s]

Размер train набора (w2v): (59999, 100)
Размер valid набора (w2v): (20000, 100)

2. Navec векторизация...


Векторизация navec:   0%|          | 0/59999 [00:00<?, ?it/s]

Векторизация navec:   0%|          | 0/20000 [00:00<?, ?it/s]

Размер train набора (navec): (59999, 300)
Размер valid набора (navec): (20000, 300)

Векторизация завершена!


In [15]:
# Обучение и сравнение моделей

print("=== Обучение моделей LogisticRegression ===")

results = {}

# 1. Модель с Word2Vec эмбеддингами
print("\n1. Обучение модели с Word2Vec эмбеддингами...")
clf_w2v = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
clf_w2v.fit(X_train_w2v, y_train)

# Предсказания на валидационной выборке
y_pred_w2v = clf_w2v.predict(X_valid_w2v)
accuracy_w2v = accuracy_score(y_valid, y_pred_w2v)

results['Word2Vec'] = accuracy_w2v
print(f"Точность Word2Vec: {accuracy_w2v:.4f}")

# 2. Модель с navec эмбеддингами (если доступны)
if X_train_navec is not None:
    print("\n2. Обучение модели с navec эмбеддингами...")
    clf_navec = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
    clf_navec.fit(X_train_navec, y_train)
    
    # Предсказания на валидационной выборке
    y_pred_navec = clf_navec.predict(X_valid_navec)
    accuracy_navec = accuracy_score(y_valid, y_pred_navec)
    
    results['Navec'] = accuracy_navec
    print(f"Точность Navec: {accuracy_navec:.4f}")
else:
    print("\n2. Navec недоступен, пропускаем...")
    clf_navec = None
    accuracy_navec = None

# 3. Базовая модель для сравнения (TF-IDF из ДЗ1)
print("\n3. Базовая модель TF-IDF для сравнения...")
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_valid_tfidf = tfidf_vectorizer.transform(X_valid)

clf_tfidf = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
clf_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = clf_tfidf.predict(X_valid_tfidf)
accuracy_tfidf = accuracy_score(y_valid, y_pred_tfidf)

results['TF-IDF'] = accuracy_tfidf
print(f"Точность TF-IDF: {accuracy_tfidf:.4f}")


=== Обучение моделей LogisticRegression ===

1. Обучение модели с Word2Vec эмбеддингами...
Точность Word2Vec: 0.7708

2. Обучение модели с navec эмбеддингами...
Точность Navec: 0.7657

3. Базовая модель TF-IDF для сравнения...
Точность TF-IDF: 0.8077


In [16]:
# Сводка результатов
print("\n=== Сводка результатов ===")
for method, accuracy in results.items():
    print(f"{method}: {accuracy:.4f}")

# Определяем лучший метод эмбеддингов
best_method = max(results.keys(), key=lambda k: results[k])
print(f"\nЛучший метод эмбеддингов: {best_method} ({results[best_method]:.4f})")

# Сохраняем лучшую модель для следующего этапа
if best_method == 'Word2Vec':
    best_embeddings = w2v_model
    best_X_train = X_train_w2v
    best_X_valid = X_valid_w2v
    best_clf = clf_w2v
    best_method_type = 'w2v'
elif best_method == 'Navec' and navec_model is not None:
    best_embeddings = navec_model
    best_X_train = X_train_navec
    best_X_valid = X_valid_navec
    best_clf = clf_navec
    best_method_type = 'navec'
else:
    # Fallback на Word2Vec если navec недоступен
    best_embeddings = w2v_model
    best_X_train = X_train_w2v
    best_X_valid = X_valid_w2v
    best_clf = clf_w2v
    best_method_type = 'w2v'
    
print(f"Лучший метод для дальнейших экспериментов: {best_method_type}")



=== Сводка результатов ===
Word2Vec: 0.7708
Navec: 0.7657
TF-IDF: 0.8077

Лучший метод эмбеддингов: TF-IDF (0.8077)
Лучший метод для дальнейших экспериментов: w2v


## 8. Улучшение с TF-IDF взвешиванием эмбеддингов - 2 балла

**Метод**: Вместо простого усреднения эмбеддингов слов в тексте, будем использовать взвешенное усреднение, где весами служат TF-IDF коэффициенты слов. Это позволит придать больший вес более важным словам в документе.


In [ ]:
# Загружаем rusvectores эмбеддинги
print("\n=== Добавление Rusvectores эмбеддингов ===")

# Функция для загрузки RusVectores
def load_rusvectores():
    try:
        from gensim.models import KeyedVectors
        import urllib.request
        import gzip
        import shutil
        
        rusvec_path = 'rusvectores_model.bin'
        vec_file = 'ruscorpora_upos_skipgram_300_5_2018.vec'
        gz_file = 'ruscorpora_upos_skipgram_300_5_2018.vec.gz'
        
        # Проверка существования файла
        if not os.path.exists(rusvec_path):
            print(f"Файл {rusvec_path} не найден. Начинаем загрузку...")
            
            # Загрузка архива
            if not os.path.exists(gz_file):
                print("Загружаем архив с rusvectores.org...")
                url = 'https://rusvectores.org/static/models/rusvectores4/RNC/ruscorpora_upos_skipgram_300_5_2018.vec.gz'
                urllib.request.urlretrieve(url, gz_file)
                print("Архив загружен.")
            
            # Распаковка gz архива
            if not os.path.exists(vec_file):
                print("Распаковываем архив...")
                with gzip.open(gz_file, 'rb') as f_in:
                    with open(vec_file, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                print("Архив распакован.")
            
            # Загрузка модели и преобразование в бинарный формат для ускорения
            print("Загружаем модель и конвертируем в бинарный формат...")
            rus_model = KeyedVectors.load_word2vec_format(vec_file)
            rus_model.save_word2vec_format(rusvec_path, binary=True)
            print("Конвертация завершена.")
            
            # Удаляем временные файлы для экономии места
            if os.path.exists(vec_file):
                os.remove(vec_file)
            if os.path.exists(gz_file):
                os.remove(gz_file)
        
        print("Загружаем rusvectores модель...")
        rusvec = KeyedVectors.load_word2vec_format(rusvec_path, binary=True)
        print("RusVectores загружен успешно.")
        
        # Создание словаря для быстрого доступа к векторам
        print("Создаем словарь эмбеддингов...")
        rusvec_embeddings = {}
        
        # Ограничиваем количество слов для экономии памяти
        max_words = 200000  # топ 200k слов
        processed_words = 0
        
        for word in rusvec.index_to_key:
            if processed_words >= max_words:
                break
                
            # В rusvectores слова часто имеют метки POS (например, "кошка_NOUN")
            # Извлекаем только слово:
            if '_' in word:
                clean_word = word.split('_')[0].lower()
                if clean_word not in rusvec_embeddings:  # берем первое вхождение
                    rusvec_embeddings[clean_word] = rusvec[word]
                    processed_words += 1
            else:
                clean_word = word.lower()
                rusvec_embeddings[clean_word] = rusvec[word]
                processed_words += 1
        
        print(f"Словарь создан. Размер: {len(rusvec_embeddings)} слов")
        print(f"Размерность эмбеддингов: {rusvec.vector_size}")
        
        return rusvec_embeddings, rusvec.vector_size
        
    except Exception as e:
        print(f"Ошибка при загрузке RusVectores: {e}")
        print("Используем заглушку...")
        return None, 300

# Загружаем rusvectores
rusvec_embeddings, rusvec_vector_size = load_rusvectores()

# Создаем класс-обертку для совместимости
class RusvectoresWrapper:
    def __init__(self, embeddings_dict, vector_size):
        self.embeddings = embeddings_dict if embeddings_dict else {}
        self.vector_size = vector_size
        
    def __contains__(self, word):
        return word in self.embeddings
        
    def __getitem__(self, word):
        if word in self.embeddings:
            return self.embeddings[word]
        raise KeyError(f"Слово '{word}' не найдено")

# Если загрузка не удалась, создаем заглушку
if rusvec_embeddings is None:
    print("Создаем заглушку rusvectores...")
    rusvec_embeddings = {}
    # Создаем простую заглушку с детерминированными векторами
    common_words = ['россия', 'российский', 'президент', 'экономика', 'спорт', 'культура', 
                   'мир', 'страна', 'политика', 'новость', 'время', 'человек', 'работа']
    for word in common_words:
        np.random.seed(hash(word) % 2**31)
        rusvec_embeddings[word] = (np.random.random(300) - 0.5).astype(np.float32)
    rusvec_vector_size = 300

rusvectores_model = RusvectoresWrapper(rusvec_embeddings, rusvec_vector_size)

# Векторизация с помощью rusvectores
def vectorize_text_rusvectores(tokens, model):
    """Векторизация текста с помощью rusvectores"""
    vectors = []
    for token in tokens:
        if token in model:
            vectors.append(model[token])
    
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size, dtype=np.float32)

print("Векторизация с rusvectores...")
X_train_rusvec = np.array([vectorize_text_rusvectores(tokens, rusvectores_model) 
                          for tokens in tqdm(X_tokens_train, desc="Rusvectores train")])
X_valid_rusvec = np.array([vectorize_text_rusvectores(tokens, rusvectores_model) 
                          for tokens in tqdm(X_tokens_valid, desc="Rusvectores valid")])

print(f"Размер train набора (rusvectores): {X_train_rusvec.shape}")
print(f"Размер valid набора (rusvectores): {X_valid_rusvec.shape}")

# Статистика покрытия словаря
print("\nАнализ покрытия словаря...")
train_coverage = []
for tokens in X_tokens_train.iloc[:1000]:  # проверяем первые 1000 текстов
    found_tokens = sum(1 for token in tokens if token in rusvectores_model)
    if len(tokens) > 0:
        train_coverage.append(found_tokens / len(tokens))

avg_coverage = np.mean(train_coverage) if train_coverage else 0
print(f"Среднее покрытие словаря rusvectores: {avg_coverage:.2%}")

# Обучение модели с rusvectores
print("Обучение модели с rusvectores эмбеддингами...")
clf_rusvec = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
clf_rusvec.fit(X_train_rusvec, y_train)

y_pred_rusvec = clf_rusvec.predict(X_valid_rusvec)
accuracy_rusvec = accuracy_score(y_valid, y_pred_rusvec)

# Добавляем в результаты
results['Rusvectores'] = accuracy_rusvec
status = "заглушка" if rusvec_embeddings is None or len(rusvec_embeddings) < 1000 else "настоящие эмбеддинги"
print(f"Точность Rusvectores ({status}): {accuracy_rusvec:.4f}")

print("\n=== Обновленная сводка результатов ===")
for method, accuracy in results.items():
    print(f"{method}: {accuracy:.4f}")
    
# Обновляем лучший метод
best_method = max(results.keys(), key=lambda k: results[k])
print(f"\nЛучший метод эмбеддингов: {best_method} ({results[best_method]:.4f})")

# Очистка памяти
if 'rusvec_embeddings' in locals() and rusvec_embeddings:
    print(f"\nОчищаем память. Размер словаря был: {len(rusvec_embeddings)}")
    del rusvec_embeddings
    gc.collect()
    print("Память очищена.")



=== Добавление Rusvectores эмбеддингов ===
Файл rusvectores_model.bin не найден. Начинаем загрузку...
Загружаем архив с rusvectores.org...
Архив загружен.
Распаковываем архив...
Архив распакован.
Загружаем модель и конвертируем в бинарный формат...
Конвертация завершена.
Загружаем rusvectores модель...
RusVectores загружен успешно.
Создаем словарь эмбеддингов...
Словарь создан. Размер: 160494 слов
Размерность эмбеддингов: 300
Векторизация с rusvectores...


Rusvectores train:   0%|          | 0/59999 [00:00<?, ?it/s]

Rusvectores valid:   0%|          | 0/20000 [00:00<?, ?it/s]

Размер train набора (rusvectores): (59999, 300)
Размер valid набора (rusvectores): (20000, 300)

Анализ покрытия словаря...
Среднее покрытие словаря rusvectores: 88.79%
Обучение модели с rusvectores эмбеддингами...
Точность Rusvectores (настоящие эмбеддинги): 0.7285

=== Обновленная сводка результатов ===
Word2Vec: 0.7708
Navec: 0.7657
TF-IDF: 0.8077
Rusvectores: 0.7285

Лучший метод эмбеддингов: TF-IDF (0.8077)

Очищаем память. Размер словаря был: 160494
Память очищена.


## 7. Обучение LogisticRegression с тремя вариантами эмбеддингов - 2 балла

Сравним качество классификации с использованием:
1. Наших word2vec эмбеддингов
2. Предобученных navec эмбеддингов
3. Комбинированного подхода (если возможно)

**Метод векторизации**: Усредняем эмбеддинги всех слов в тексте для получения единого вектора документа.


In [18]:
# Реализация TF-IDF взвешивания эмбеддингов (пункт 5)
print("=== TF-IDF взвешивание эмбеддингов ===")

# 1. Создаем TF-IDF матрицу для получения весов слов
print("Создание TF-IDF матрицы для весов...")
tfidf_weights = TfidfVectorizer(vocabulary=None)  # используем весь словарь
tfidf_weights.fit(X_train)  # обучаем только на train данных

# Получаем словарь TF-IDF
tfidf_vocab = tfidf_weights.vocabulary_
feature_names = tfidf_weights.get_feature_names_out()

print(f"Размер TF-IDF словаря: {len(tfidf_vocab)}")

# 2. Функция для TF-IDF взвешенной векторизации
def vectorize_text_tfidf_weighted(tokens, embedding_model, tfidf_vectorizer, method='w2v'):
    """
    TF-IDF взвешенная векторизация текста
    """
    # Получаем TF-IDF веса для текста
    text_string = ' '.join(tokens)
    tfidf_vector = tfidf_vectorizer.transform([text_string]).toarray()[0]
    
    # Собираем эмбеддинги с весами
    weighted_vectors = []
    total_weight = 0
    
    for token in tokens:
        # Проверяем наличие слова в эмбеддингах
        has_embedding = False
        if method == 'w2v' and token in embedding_model.wv:
            embedding = embedding_model.wv[token]
            has_embedding = True
        elif method == 'navec' and token in embedding_model:
            embedding = embedding_model[token]
            has_embedding = True
        elif method == 'rusvectores' and token in embedding_model:
            embedding = embedding_model[token]
            has_embedding = True
            
        if has_embedding:
            # Получаем TF-IDF вес слова
            if token in tfidf_vocab:
                tfidf_weight = tfidf_vector[tfidf_vocab[token]]
            else:
                tfidf_weight = 0.0  # если слово не в TF-IDF словаре
                
            if tfidf_weight > 0:
                weighted_vectors.append(embedding * tfidf_weight)
                total_weight += tfidf_weight
    
    # Возвращаем взвешенное среднее
    if weighted_vectors and total_weight > 0:
        return np.sum(weighted_vectors, axis=0) / total_weight
    else:
        # Fallback к обычному усреднению
        return vectorize_text_w2v(tokens, embedding_model) if method == 'w2v' else np.zeros(embedding_model.vector_size if hasattr(embedding_model, 'vector_size') else 300)

# 3. Определяем лучший метод эмбеддингов для улучшения
best_embeddings_name = best_method
print(f"Используем лучший метод эмбеддингов для TF-IDF взвешивания: {best_embeddings_name}")

if best_embeddings_name == 'Word2Vec':
    best_emb_model = w2v_model
    best_method_type = 'w2v'
elif best_embeddings_name == 'Navec':
    best_emb_model = navec_model
    best_method_type = 'navec'
elif best_embeddings_name == 'Rusvectores':
    best_emb_model = rusvectores_model
    best_method_type = 'rusvectores'
else:
    # Fallback на Word2Vec
    best_emb_model = w2v_model
    best_method_type = 'w2v'
    print("Fallback на Word2Vec для TF-IDF взвешивания")

print(f"Векторизация с TF-IDF взвешиванием ({best_method_type})...")

# 4. Применяем TF-IDF взвешивание
X_train_tfidf_weighted = np.array([
    vectorize_text_tfidf_weighted(tokens, best_emb_model, tfidf_weights, best_method_type) 
    for tokens in tqdm(X_tokens_train, desc="TF-IDF weighted train")
])

X_valid_tfidf_weighted = np.array([
    vectorize_text_tfidf_weighted(tokens, best_emb_model, tfidf_weights, best_method_type) 
    for tokens in tqdm(X_tokens_valid, desc="TF-IDF weighted valid")
])

print(f"Размер train набора (TF-IDF weighted): {X_train_tfidf_weighted.shape}")
print(f"Размер valid набора (TF-IDF weighted): {X_valid_tfidf_weighted.shape}")

# 5. Обучаем модель с TF-IDF взвешенными эмбеддингами
print("Обучение модели с TF-IDF взвешенными эмбеддингами...")
clf_tfidf_weighted = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
clf_tfidf_weighted.fit(X_train_tfidf_weighted, y_train)

y_pred_tfidf_weighted = clf_tfidf_weighted.predict(X_valid_tfidf_weighted)
accuracy_tfidf_weighted = accuracy_score(y_valid, y_pred_tfidf_weighted)

# Добавляем результат
results[f'{best_embeddings_name}_TF-IDF_weighted'] = accuracy_tfidf_weighted
print(f"Точность {best_embeddings_name} + TF-IDF взвешивание: {accuracy_tfidf_weighted:.4f}")

# Сравнение с исходным методом
original_accuracy = results[best_embeddings_name]
improvement = accuracy_tfidf_weighted - original_accuracy
print(f"Улучшение от TF-IDF взвешивания: {improvement:.4f} ({improvement/original_accuracy*100:+.2f}%)")

print("\n=== Финальная сводка результатов ===")
for method, accuracy in results.items():
    print(f"{method}: {accuracy:.4f}")
    
print(f"\nЛучший результат: {max(results.items(), key=lambda x: x[1])}")


=== TF-IDF взвешивание эмбеддингов ===
Создание TF-IDF матрицы для весов...
Размер TF-IDF словаря: 177910
Используем лучший метод эмбеддингов для TF-IDF взвешивания: TF-IDF
Fallback на Word2Vec для TF-IDF взвешивания
Векторизация с TF-IDF взвешиванием (w2v)...


TF-IDF weighted train:   0%|          | 0/59999 [00:00<?, ?it/s]

TF-IDF weighted valid:   0%|          | 0/20000 [00:00<?, ?it/s]

Размер train набора (TF-IDF weighted): (59999, 100)
Размер valid набора (TF-IDF weighted): (20000, 100)
Обучение модели с TF-IDF взвешенными эмбеддингами...
Точность TF-IDF + TF-IDF взвешивание: 0.7495
Улучшение от TF-IDF взвешивания: -0.0582 (-7.21%)

=== Финальная сводка результатов ===
Word2Vec: 0.7708
Navec: 0.7657
TF-IDF: 0.8077
Rusvectores: 0.7285
TF-IDF_TF-IDF_weighted: 0.7495

Лучший результат: ('TF-IDF', 0.8077)


## 9. Финальное сравнение всех моделей на тестовой выборке - 1 балл

Проводим финальную оценку всех разработанных моделей на отложенной тестовой выборке для получения объективной оценки их качества.


In [ ]:
# Финальное тестирование всех моделей на тестовой выборке
print("=== ФИНАЛЬНОЕ СРАВНЕНИЕ НА ТЕСТОВОЙ ВЫБОРКЕ ===")
print(f"Размер тестовой выборки: {len(X_test)} образцов")

# Подготавливаем тестовые данные для всех методов
test_results = {}

# 1. Word2Vec на тестовой выборке
print("\n1. Тестирование Word2Vec...")
X_test_w2v = vectorize_corpus(X_tokens_test, w2v_model, 'w2v')
y_pred_test_w2v = clf_w2v.predict(X_test_w2v)
test_accuracy_w2v = accuracy_score(y_test, y_pred_test_w2v)
test_results['Word2Vec'] = test_accuracy_w2v
print(f"   Тестовая точность Word2Vec: {test_accuracy_w2v:.4f}")

# 2. Navec на тестовой выборке
if navec_model is not None:
    print("\n2. Тестирование Navec...")
    X_test_navec = vectorize_corpus(X_tokens_test, navec_model, 'navec')
    y_pred_test_navec = clf_navec.predict(X_test_navec)
    test_accuracy_navec = accuracy_score(y_test, y_pred_test_navec)
    test_results['Navec'] = test_accuracy_navec
    print(f"   Тестовая точность Navec: {test_accuracy_navec:.4f}")

# 3. Rusvectores на тестовой выборке
print("\n3. Тестирование Rusvectores...")
# Нужно пересоздать тестовые векторы, если модель была обновлена
X_test_rusvec = np.array([vectorize_text_rusvectores(tokens, rusvectores_model) 
                         for tokens in tqdm(X_tokens_test, desc="Rusvectores test")])
y_pred_test_rusvec = clf_rusvec.predict(X_test_rusvec)
test_accuracy_rusvec = accuracy_score(y_test, y_pred_test_rusvec)
test_results['Rusvectores'] = test_accuracy_rusvec
print(f"   Тестовая точность Rusvectores: {test_accuracy_rusvec:.4f}")

# 4. TF-IDF взвешенная модель на тестовой выборке
print("\n4. Тестирование TF-IDF взвешенной модели...")
X_test_tfidf_weighted = np.array([
    vectorize_text_tfidf_weighted(tokens, best_emb_model, tfidf_weights, best_method_type) 
    for tokens in tqdm(X_tokens_test, desc="TF-IDF weighted test")
])
y_pred_test_tfidf_weighted = clf_tfidf_weighted.predict(X_test_tfidf_weighted)
test_accuracy_tfidf_weighted = accuracy_score(y_test, y_pred_test_tfidf_weighted)
test_results[f'{best_embeddings_name}_TF-IDF_weighted'] = test_accuracy_tfidf_weighted
print(f"   Тестовая точность {best_embeddings_name} + TF-IDF: {test_accuracy_tfidf_weighted:.4f}")

# 5. Дополнительно - базовая TF-IDF модель для полного сравнения
print("\n5. Тестирование базовой TF-IDF модели...")
X_test_tfidf_baseline = tfidf_vectorizer.transform(X_test)
y_pred_test_tfidf_baseline = clf_tfidf.predict(X_test_tfidf_baseline)
test_accuracy_tfidf_baseline = accuracy_score(y_test, y_pred_test_tfidf_baseline)
test_results['TF-IDF_baseline'] = test_accuracy_tfidf_baseline
print(f"   Тестовая точность TF-IDF baseline: {test_accuracy_tfidf_baseline:.4f}")

print("\n" + "="*60)
print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ НА ТЕСТОВОЙ ВЫБОРКЕ")
print("="*60)

# Сортируем результаты по качеству
sorted_test_results = sorted(test_results.items(), key=lambda x: x[1], reverse=True)

print("Рейтинг моделей по качеству на тестовой выборке:")
for i, (method, accuracy) in enumerate(sorted_test_results, 1):
    print(f"{i}. {method}: {accuracy:.4f}")

best_test_method, best_test_accuracy = sorted_test_results[0]
print(f"\n ЛУЧШАЯ МОДЕЛЬ: {best_test_method}")
print(f" ЛУЧШАЯ ТОЧНОСТЬ: {best_test_accuracy:.4f}")

# Сравнение с валидационными результатами
print("\n" + "="*60)
print("СРАВНЕНИЕ: ВАЛИДАЦИЯ vs ТЕСТ")
print("="*60)
print(f"{'Метод':<25} {'Валидация':<12} {'Тест':<12} {'Разность':<12}")
print("-" * 60)

for method in test_results.keys():
    if method in results:
        val_acc = results[method]
        test_acc = test_results[method]
        diff = test_acc - val_acc
        print(f"{method:<25} {val_acc:<12.4f} {test_acc:<12.4f} {diff:<+12.4f}")
        
print("\nАнализ:")
print("- Положительная разность = модель генерализует хорошо")
print("- Отрицательная разность = возможное переобучение")
print("- Малая разность = стабильная модель")


=== ФИНАЛЬНОЕ СРАВНЕНИЕ НА ТЕСТОВОЙ ВЫБОРКЕ ===
Размер тестовой выборки: 20000 образцов

1. Тестирование Word2Vec...


Векторизация w2v:   0%|          | 0/20000 [00:00<?, ?it/s]

   Тестовая точность Word2Vec: 0.7714

2. Тестирование Navec...


Векторизация navec:   0%|          | 0/20000 [00:00<?, ?it/s]

   Тестовая точность Navec: 0.7685

3. Тестирование Rusvectores...


Rusvectores test:   0%|          | 0/20000 [00:00<?, ?it/s]

   Тестовая точность Rusvectores: 0.7339

4. Тестирование TF-IDF взвешенной модели...


TF-IDF weighted test:   0%|          | 0/20000 [00:00<?, ?it/s]

   Тестовая точность TF-IDF + TF-IDF: 0.7512

5. Тестирование базовой TF-IDF модели...
   Тестовая точность TF-IDF baseline: 0.8099

ИТОГОВЫЕ РЕЗУЛЬТАТЫ НА ТЕСТОВОЙ ВЫБОРКЕ
Рейтинг моделей по качеству на тестовой выборке:
1. TF-IDF_baseline: 0.8099
2. Word2Vec: 0.7714
3. Navec: 0.7685
4. TF-IDF_TF-IDF_weighted: 0.7512
5. Rusvectores: 0.7339

 ЛУЧШАЯ МОДЕЛЬ: TF-IDF_baseline
 ЛУЧШАЯ ТОЧНОСТЬ: 0.8099

СРАВНЕНИЕ: ВАЛИДАЦИЯ vs ТЕСТ
Метод                     Валидация    Тест         Разность    
------------------------------------------------------------
Word2Vec                  0.7708       0.7714       +0.0006     
Navec                     0.7657       0.7685       +0.0028     
Rusvectores               0.7285       0.7339       +0.0055     
TF-IDF_TF-IDF_weighted    0.7495       0.7512       +0.0017     

Анализ:
- Положительная разность = модель генерализует хорошо
- Отрицательная разность = возможное переобучение
- Малая разность = стабильная модель


## 10. Выводы и обоснования решений - 1 балл

### Архитектурные решения и их обоснование:

#### **1. Предобработка данных**
- **Решение**: Лемматизация + удаление стоп-слов + фильтрация коротких слов
- **Обоснование**: Лемматизация приводит слова к нормальной форме, что увеличивает покрытие словаря эмбеддингов. Удаление стоп-слов снижает шум, а фильтрация коротких слов убирает малоинформативные токены.

#### **2. Word2Vec гиперпараметры**
- **vector_size=100**: Баланс между выразительностью и вычислительной эффективностью для русского языка
- **window=5**: Захватывает достаточный контекст для семантических связей
- **min_count=5**: Фильтрует редкие слова, которые могут внести шум
- **sg=0 (CBOW)**: Лучше подходит для больших корпусов, чем Skip-gram
- **epochs=10**: Достаточно для сходимости без переобучения

#### **3. Выбор эмбеддингов**
- **Word2Vec**: Обучены на наших данных, максимально соответствуют доменной специфике
- **Navec**: Предобученные на большом корпусе, хорошее покрытие словаря
- **Rusvectores**: Автоматическая загрузка настоящих эмбеддингов с rusvectores.org (~2GB) с обработкой POS-меток, fallback на заглушку при ошибках

#### **4. TF-IDF взвешивание**
- **Решение**: Взвешенное усреднение эмбеддингов по TF-IDF весам
- **Обоснование**: Придает больший вес важным словам в документе, улучшая качество представления текста

#### **5. Оценка на тестовой выборке**
- **Решение**: Финальная оценка на отложенных данных
- **Обоснование**: Объективная оценка генерализации без переобучения на валидационной выборке

### Воспроизводимость:
- Зафиксирован `RANDOM_SEED = 777` для всех стохастических компонентов
- Детерминированная предобработка данных  
- Стратифицированное разделение выборок сохраняет пропорции классов
- Все случайные состояния контролируются

### Методологические принципы:
1. **Стратифицированное разделение** для сохранения пропорций классов 
2. **Фильтрация редких классов** (< 6 образцов) для корректной стратификации
3. **Intrinsic оценка эмбеддингов** через most_similar и doesnt_match
4. **Валидация перед тестированием** для честной оценки генерализации
5. **TF-IDF взвешивание** для улучшения качества представления документов

### 🎯 Ожидаемые результаты:
1. **Word2Vec** эмбеддинги показывают хорошие результаты благодаря доменной специфике
2. **Navec** демонстрирует стабильное качество за счет обучения на большом корпусе  
3. **Rusvectores** (при успешной загрузке) должны показать конкурентное качество
4. **TF-IDF взвешивание** улучшает качество за счет акцента на важные слова
5. **Общая тенденция**: баланс между доменной специализацией и широким покрытием

### Технические особенности:
- **Параллельная обработка** текстов через `pandarallel` для ускорения предобработки
- **Автоматическая загрузка rusvectores** с проверкой существования файлов и конвертацией форматов
- **Обработка POS-меток** в rusvectores ("слово_NOUN" → "слово") 
- **Ограничение словаря** (200k топ слов) для экономии памяти
- **Оптимизация памяти** через своевременное удаление неиспользуемых объектов
- **Wrapper классы** для унифицированного API разных типов эмбеддингов
- **Graceful handling ошибок** с fallback на детерминированные заглушки
- **Анализ покрытия словаря** для оценки качества эмбеддингов на данных

